In [1]:
%pip install tensorflow numpy matplotlib pandas scikit-learn seaborn


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"

import numpy as np
import matplotlib.pyplot as plt
import keras
from keras import Model, Input, layers, losses, activations, ops, callbacks, models
from sklearn.model_selection import train_test_split
import pandas as pd
import json
import glob
from datetime import timedelta, datetime


In [5]:
@keras.saving.register_keras_serializable()
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.seed_generator = keras.random.SeedGenerator(1337)

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = ops.shape(z_mean)[0]
        dim = ops.shape(z_mean)[1]
        epsilon = keras.random.normal(shape=(batch, dim), seed=self.seed_generator)
        return z_mean + ops.exp(0.5 * z_log_var) * epsilon

encoder = models.load_model("models/encoder.keras", compile=False)
decoder = models.load_model("models/decoder.keras", compile=False)
vae = models.load_model("models/vae.keras", compile=False)

In [32]:
def create_groups(df, day_no):
    day_df = df[df['timestamp'].dt.weekday == day_no]
    day_df.set_index('timestamp', inplace=True)
    
    mylist = list(day_df.groupby(day_df.index.date))
    
    if len(mylist) == 0:
        return create_groups(df, np.random.randint(0,6))
    
    return mylist


def generate_csv(userId: int):
    df_steps = pd.read_csv('fitabase/data/hourlySteps_merged.csv')
    df_steps = df_steps[df_steps['Id'] == userId]
    df_steps['timestamp'] = pd.to_datetime(df_steps['ActivityHour'], format="%m/%d/%Y %I:%M:%S %p")

    df_sleep = pd.read_csv("fitabase/data/minuteSleep_merged.csv")
    df_sleep = df_sleep[df_sleep['Id'] == userId]
    df_sleep['timestamp'] = pd.to_datetime(df_sleep['date'], format="%m/%d/%Y %I:%M:%S %p")

    df_hr = pd.read_csv('fitabase/data/heartrate_seconds_merged.csv')
    df_hr = df_hr[df_hr['Id'] == userId]
    df_hr['timestamp'] = pd.to_datetime(df_hr['Time'], format="%m/%d/%Y %I:%M:%S %p")
    df_hr['normal_val'] = df_hr['Value'].astype('float32') / 250.0

    # The day of the week with Monday=0, Sunday=6
    steps_by_hour = {d: {
                        h : df_steps[(df_steps["timestamp"].dt.hour == h) & (df_steps["timestamp"].dt.weekday == d)]['StepTotal']
                        for h in range(24)} for d in range(7)
                    }
    sleep_by_wday = {d: create_groups(df_sleep, d) for d in range(7) }
    hr_by_wday = {i: df_hr[df_hr["timestamp"].dt.weekday == i]['normal_val'] for i in range(7)}

    date_range = pd.date_range(start="2025-01-01", end="2025-08-31", freq="D")
    df_data = pd.DataFrame(columns=["datetime", "reading", "data_type"])

    for day in date_range:
        ref_step = steps_by_hour[day.weekday()]
        hour_range = pd.date_range(start=day, end=day + pd.Timedelta(days=1), freq="h", inclusive='left')

        value_list = [np.random.normal(loc=ref_step[h].mean(), scale=ref_step[h].std(), size=1)[0]  for h in range(24)]
        value_list = np.nan_to_num(value_list)
        value_list = np.clip(value_list, a_min=0, a_max=None)
        
        zipped = list(zip(hour_range, value_list))
        df0 = pd.DataFrame(zipped, columns=["datetime", "reading"])
        df0['data_type'] = 2

        ref_hr = hr_by_wday[day.weekday()]
        two_min_range = pd.date_range(start=day, end=day + pd.Timedelta(days=1), freq="2min", inclusive='left')
        
        hr_random = np.random.normal(loc=ref_hr.mean(), scale=ref_hr.std(),size=(1, 720))
        hr_random = vae.predict(hr_random)
        hr_random = hr_random[0]*250

        zipped = list(zip(two_min_range, hr_random))
        df1 = pd.DataFrame(zipped, columns=["datetime", "reading"])
        df1['data_type'] = 3

        ref_sleep = sleep_by_wday[day.weekday()]
        ran_idx = np.random.randint(0, len(ref_sleep))


        ref_sleep = ref_sleep[ran_idx][1]
        ref_sleep = ref_sleep.reset_index()
        ref_sleep['timestamp'] = ref_sleep['timestamp'].apply(lambda dt: datetime.combine(day.date(), dt.time()))
        ref_sleep = ref_sleep.rename(columns={'timestamp': 'datetime', 'value': 'reading'})
        df2 = ref_sleep[['datetime', 'reading']]
        df2['data_type'] = 4

        df_data = pd.concat([df_data, df0, df1, df2], axis=0, ignore_index=True)

    df_meteo = pd.read_csv('meteo/data/ogd-smn_klo_h_recent.csv', delimiter=';')
    df_meteo['datetime'] = pd.to_datetime(df_meteo['reference_timestamp'], format="%d.%m.%Y %H:%M")

    for idx, type in enumerate(['tre200h0', 'rre150h0', 'pp0qffh0']):
        df_file = df_meteo[['datetime', type]]
        df_file = df_file.rename(columns={type: "reading"})
        df_file['data_type'] = 10 + idx
        df_data = pd.concat([df_data, df_file], axis=0, ignore_index=True)

    df_pollen = pd.read_csv('meteo/data/ogd-pollen_pzh_h_recent.csv', delimiter=';')
    df_pollen['datetime'] = pd.to_datetime(df_meteo['reference_timestamp'], format="%d.%m.%Y %H:%M")
    df_pollen = df_pollen[['datetime', 'khpoach0']]
    df_pollen = df_pollen.rename(columns={'khpoach0': "reading"})
    df_pollen['data_type'] = 9
    
    df_data = pd.concat([df_data, df_pollen], axis=0, ignore_index=True)

    df_data['subject_id'] = userId
    df_data.to_csv(f"csv/{userId}.csv", columns=['subject_id', 'datetime', 'reading', 'data_type'], index=False, header=False)



In [33]:
subject_ids = [
    2347167796,
    4020332650,
    4558609924,
    5553957443,
    5577150313,
    6962181067,
    8792009665,
]

for user in subject_ids:
    generate_csv(user)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step


/tmp/ipykernel_3712410/3815341799.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['data_type'] = 4
/tmp/ipykernel_3712410/3815341799.py:72: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_data = pd.concat([df_data, df0, df1, df2], axis=0, ignore_index=True)
/tmp/ipykernel_3712410/3815341799.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


/tmp/ipykernel_3712410/3815341799.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['data_type'] = 4
/tmp/ipykernel_3712410/3815341799.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['data_type'] = 4
/tmp/ipykernel_3712410/3815341799.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step


...

/tmp/ipykernel_3712410/3815341799.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['data_type'] = 4
/tmp/ipykernel_3712410/3815341799.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['data_type'] = 4


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step


/tmp/ipykernel_3712410/3815341799.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['data_type'] = 4
/tmp/ipykernel_3712410/3815341799.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['data_type'] = 4


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step


/tmp/ipykernel_3712410/3815341799.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['data_type'] = 4
